# Reproduce Results from Perdikaris with QUEENS

The purpose of this notebook is the reproduction of some of the results presented in [Perdikaris et al. 2017](http://rspa.royalsocietypublishing.org/content/473/2198/20160751). The code published along with the original publication can be found [here](https://github.com/paraklas/NARGP).

## 1D Sinus function

The first example is the following multi-fidelity expression:

The low-fidelity version is defined as:

$f_l(x) = sin(8\pi x)$

And the high fidelity version as:

$f_h(x) = (x- \sqrt(2)f_l^2(x))$

Below, we will build multi-fidelity surrogate models using the nonlinear autoregressive scheme put forth in [[1](http://rspa.royalsocietypublishing.org/content/473/2198/20160751)] and for comparison the original autoregressive scheme proposed by Kennedy and O'Hagan in [[2](https://academic.oup.com/biomet/article/87/1/1/221217)]. The respective surrogate models will be referred to as NARGP and ICM.


### Input File

In [1]:
# Define multi-fidelity model
options = { 
    "output_dir" : "/Users/jonas/work/adco/queens_code/pqueens/example_input_files/output",
    "experiment-name" : "perdikaris_icm_test",
    "database"  :{
      "address"   : "localhost:27017"
    },
    "method": {
      "method_name": "monte_carlo",
      "method_options" :{
        "seed" : 44,
        "num_samples" : 200,
        "model" : "mf_gp_icm_surrogate_model"
      }
    },
    "mf_gp_icm_surrogate_model" : {
      "type" : "datafit_surrogate_model_mf",
      "interface" : "approximation_interface",
      "parameters" : "parameters",
      "subordinate_model" : "truth_model",
      "subordinate_iterator" : "thruth_model_iterator"
    },
    "approximation_interface" : {
      "type" : "approximation_interface_mf",
      "approximation" : "mf_gp_regression"
    },
    "mf_gp_regression" : {
      "type" : "mf_icm_gp_approximation_gpy",
    },
    "mf_gp_nargp_surrogate_model" : {
      "type" : "datafit_surrogate_model_mf",
      "interface" : "approximation_interface2",
      "parameters" : "parameters",
      "subordinate_model" : "truth_model",
      "subordinate_iterator" : "thruth_model_iterator"
    },
    "approximation_interface2" : {
      "type" : "approximation_interface_mf",
      "approximation" : "mf_gp_regression2"
    },
    "mf_gp_regression2" : {
      "type" : "mf_nar_gp_approximation_gpy",
    },
    "thruth_model_iterator" : {
      "method_name": "lhs_mf",
      "method_options" :{
        "seed" : 42,
        "num_samples" : [40, 30],
        "num_iterations" :10,
        "mode" : "nested"
      }
    },
    "truth_model" :{
      "type" : "multi_fidelity_model",
      "model_hierarchy" : ["lofi_sin", "hifi_sin"],
      "eval_cost_per_level" :[1, 1],
      "parameters" : "parameters"
    },
    "hifi_sin" : {
      "type" : "simulation_model",
      "interface" : "interface_hifi"
    },
    "lofi_sin" : {
      "type" : "simulation_model",
      "interface" : "interface_lofi"
    },
    "interface_lofi" : {
      "type" : "direct_python_interface",
      "main_file" : "/Users/jonas/work/adco/queens_code/pqueens/pqueens/example_simulator_functions/perdikaris_1dsin_lofi.py"
    },
    "interface_hifi" : {
      "type" : "direct_python_interface",
      "main_file" : "/Users/jonas/work/adco/queens_code/pqueens/pqueens/example_simulator_functions/perdikaris_1dsin_hifi.py"
    },
    "parameters" : {
        "x" : {
            "type" : "FLOAT",
            "size" : 1,
            "min"  : 0,
            "max"  : 1,
            "distribution" : "uniform",
            "distribution_parameter" : [0,1]
        }
    }
}



### Run QUEENS


In [2]:
#from pqueens.iterators.iterator import Iterator
# build iterator
#my_iterator = Iterator.from_config_create_iterator(options)
# perform analysis
#my_iterator.run()


### Build step by step

In [4]:
from pqueens.models.model import Model
import numpy as np

np.random.seed(options["method"]["method_options"]["seed"])
num_samples = options["method"]["method_options"]["num_samples"]


# create surrogate models 
icm_model = Model.from_config_create_model("mf_gp_icm_surrogate_model", options)
nargp_model = Model.from_config_create_model("mf_gp_nargp_surrogate_model", options)

distribution_info = nargp_model.get_parameter_distribution_info()
numparams = len(distribution_info)
samples = np.zeros((num_samples, numparams))

i = 0
for distribution in distribution_info:
    # get appropriate random number generator
    random_number_generator = getattr(np.random, distribution['distribution'])
    # make a copy
    my_args = list(distribution['distribution_parameter'])
    my_args.extend([num_samples])
    samples [:, i] = random_number_generator(*my_args)
    i += 1

# update model
icm_model.update_model_from_sample_batch(samples)
nargp_model.update_model_from_sample_batch(samples)

/Applications/anaconda/envs/test/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


### Compute Results

In [5]:
from pqueens.example_simulator_functions.perdikaris_1dsin_hifi import perdikaris_1dsin_hifi


ref_outputs = perdikaris_1dsin_hifi(samples)

icm_outputs = icm_model.evaluate()
nargp_outputs = nargp_model.evaluate()



/Users/jonas/work/adco/queens_code/pqueens/pqueens/iterators/scale_samples.py:44: UserWarning: Bounds of parameters are ignored when using                 scale_samples
  scale_samples")


hifi_index 0 len(self.__model_sequence 2
hifi_index 1 len(self.__model_sequence 2
Size of inputs in LHS(40, 1)
Inputs [[0.14489482]
 [0.86987965]
 [0.03753791]
 [0.58039553]
 [0.95063377]
 [0.74093575]
 [0.76815228]
 [0.65129204]
 [0.81290751]
 [0.264457  ]
 [0.22734955]
 [0.40076251]
 [0.0912491 ]
 [0.4259337 ]
 [0.71351588]
 [0.35716353]
 [0.48400477]
 [0.33856612]
 [0.62713369]
 [0.47056501]
 [0.68828387]
 [0.92696141]
 [0.06995738]
 [0.99906621]
 [0.53805608]
 [0.11754917]
 [0.17225013]
 [0.38977083]
 [0.88177081]
 [0.50317651]
 [0.31163995]
 [0.18344988]
 [0.61557226]
 [0.7993963 ]
 [0.83307391]
 [0.91097429]
 [0.01278356]
 [0.27589856]
 [0.20938957]
 [0.56924984]]
Size of outputs (40, 1)
Outputs [[-0.47943544]
 [ 0.12833341]
 [ 0.80957662]
 [ 0.9005499 ]
 [-0.94601395]
 [-0.22584417]
 [ 0.44055485]
 [-0.61374163]
 [ 0.99994755]
 [ 0.35540202]
 [-0.53901558]
 [-0.60318027]
 [ 0.75012606]
 [-0.95804556]
 [-0.79374774]
 [ 0.43341554]
 [-0.39126344]
 [ 0.79297912]
 [-0.0535997 ]
 [-0

reconstraining parameters gp.mixed_noise.Gaussian_noise_0.variance
reconstraining parameters gp.mixed_noise.Gaussian_noise_1.variance


Optimization restart 1/30, f = -128.64108163461498
Optimization restart 2/30, f = -128.64108033501415
Optimization restart 3/30, f = 56.45935061629093
Optimization restart 4/30, f = 56.45935068474803
Optimization restart 5/30, f = 56.459351217692166
Optimization restart 6/30, f = 56.45935056295836
Optimization restart 7/30, f = 56.459350590115136
Optimization restart 8/30, f = 56.459350612761384
Optimization restart 9/30, f = 56.45935065568544
Optimization restart 10/30, f = 56.45935102111286
Optimization restart 11/30, f = 56.45935095696238
Optimization restart 12/30, f = 56.45935059709389
Optimization restart 13/30, f = 56.45935074675773
Optimization restart 14/30, f = 56.45935061276289
Optimization restart 15/30, f = 56.45935063391706
Optimization restart 16/30, f = 56.45935057831631
Optimization restart 17/30, f = 56.45935061063347
Optimization restart 18/30, f = 56.4593505538528
Optimization restart 19/30, f = 56.459350829239625
Optimization restart 20/30, f = 56.45935067490633
Op

 /Users/jonas/work/adco/queens_code/pqueens/pqueens/iterators/scale_samples.py:44: UserWarning:Bounds of parameters are ignored when using                 scale_samples
reconstraining parameters GP_regression.Gaussian_noise.variance


...

Optimization restart 1/30, f = -120.03461457748301
Optimization restart 2/30, f = -120.0635851421568
Optimization restart 3/30, f = -120.0422116170522
Optimization restart 4/30, f = -120.04778299069673
Optimization restart 5/30, f = -120.00457610199032
Optimization restart 6/30, f = -120.02922118287012
Optimization restart 7/30, f = -120.03421229957291
Optimization restart 8/30, f = -120.04340996442251
Optimization restart 9/30, f = -119.91079066152031


 /Applications/anaconda/envs/test/lib/python3.6/site-packages/GPy/kern/src/stationary.py:136: RuntimeWarning:overflow encountered in square
 /Applications/anaconda/envs/test/lib/python3.6/site-packages/GPy/kern/src/stationary.py:137: RuntimeWarning:invalid value encountered in add


Optimization restart 10/30, f = -120.07391489984377
Optimization restart 11/30, f = -120.05181978375538
Optimization restart 12/30, f = -120.06196730090083
Optimization restart 13/30, f = -120.03406660715603
Optimization restart 14/30, f = -120.02074027684483
Optimization restart 15/30, f = -120.00735973579637
Optimization restart 16/30, f = -120.03719554341872
Optimization restart 17/30, f = -120.03778526094145
Optimization restart 18/30, f = -120.00380767258233
Optimization restart 19/30, f = -120.01194934508698


 /Applications/anaconda/envs/test/lib/python3.6/site-packages/GPy/kern/src/stationary.py:167: RuntimeWarning:overflow encountered in true_divide
 /Applications/anaconda/envs/test/lib/python3.6/site-packages/GPy/kern/src/rbf.py:43: RuntimeWarning:overflow encountered in square


Optimization restart 20/30, f = -120.03784498023182
Optimization restart 21/30, f = -120.04564908486228
Optimization restart 22/30, f = -120.0467964349738
Optimization restart 23/30, f = -120.03262324582971
Optimization restart 24/30, f = -120.0244893287751
Optimization restart 25/30, f = -120.02746229545326
Optimization restart 26/30, f = -120.03250860324492
Optimization restart 27/30, f = -120.02919471161141
Optimization restart 28/30, f = -120.00691177893574
Optimization restart 29/30, f = -120.05033915648525
Optimization restart 30/30, f = -120.02087288289674


In [6]:
def sort_ascending(array_1, array_2):
    """ Sort both arrays in increasing order based on array_1"""
    data = np.hstack((array_1,array_2))
    data = data[data[:,0].argsort()]
    return data[:,0], data[:,1]
    

### Plot results

In [7]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.plotly as py
from plotly.graph_objs import *
import plotly.graph_objs as go
import numpy as np
init_notebook_mode(connected=True)

x_sorted, y_sorted = sort_ascending(samples,ref_outputs)
# Create a trace
reference = go.Scatter(
    x = x_sorted,
    y = y_sorted,
    mode = 'markers-line',
    name = 'Hi-fi reference'
)

#x_sorted, y_sorted = sort_ascending(samples,nargp_outputs)
icm = go.Scatter(
    x = samples[:,0],
    y = icm_outputs[:,0],
    mode = 'markers',
    name = 'ICM'
)
nargp = go.Scatter(
    x = samples[:,0],
    y = nargp_outputs[:,0],
    mode = 'markers',
    name = 'NARGP'
)



layout = dict(title = 'Multi-Fidelity Surrogate Modelling',
              xaxis = dict(title = 'Input'),
              yaxis = dict(title = 'Output'),
              )

fig = Figure(data=[reference, icm, nargp], layout=layout)


iplot(fig)

#print(outputs)